In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2
import os
import glob

base_train_dir = './data/train'
base_test_dir = './data/test'
base_valid_dir = './data/valid'

IMAGE_SIZE = 1280
BATCH_SIZE = 8
NUM_EPOCHS = 20

file1 = open('labels.txt', 'r')
lines = file1.readlines()
counter = 0
checkpoint_period = 10

with open('corrected_labels.txt', 'a') as f:
    f.write('{')

for line in lines:
    with open('corrected_labels.txt', 'a') as f:
        f.write(f"'{line.strip()}': {counter}, ")
    counter+=1

with open('corrected_labels.txt', 'a') as f:
    f.write('}')


builder = tfds.folder_dataset.ImageFolder('data/')
print(builder.info)


tfds.core.DatasetInfo(
    name='image_folder',
    full_name='image_folder/1.0.0',
    description="""
    Generic image classification dataset.
    """,
    homepage='https://www.tensorflow.org/datasets/catalog/image_folder',
    data_path='data/',
    download_size=Unknown size,
    dataset_size=Unknown size,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=70),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=91, num_shards=1>,
        'train': <SplitInfo num_examples=732, num_shards=1>,
        'valid': <SplitInfo num_examples=64, num_shards=1>,
    },
    citation="""""",
)


In [ ]:
def generate_batches(path, batchSize):
    while True:
        files = glob.glob(path+'/*/*jpg')
        for f in range(0, len(files), batchSize):
            x = []
            y = []
            for i in range(f, f+batchSize):
                if i < len(files):
                    img = cv2.imread(files[i])
                    x.append(cv2.resize(img, (224, 224)))
                    y.append(int(files[i].split('/')[1]))
            yield(np.array(x), to_categorical(y, num_classes=70))
                    

In [10]:
raw_train = builder.as_dataset(split='train', shuffle_files=True)
raw_test = builder.as_dataset(split='test', shuffle_files=True)
raw_valid = builder.as_dataset(split='valid', shuffle_files=True)

In [4]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape = IMG_SHAPE,
    include_top = False,
)

In [5]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3, activation= 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(70, activation='softmax')
])

In [6]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
checkpoint = ModelCheckpoint("logs/weights.h5", monitor='loss', save_best_only=True, period=checkpoint_period)
model.fit_generator(generate_batches(base_train_dir, BATCH_SIZE), epochs=NUM_EPOCHS, steps_per_epoch=len(files)//BATCH_SIZE, verbose=1, callbacks=[checkpoint])

In [11]:
epochs = 20
history = model.fit(
    raw_train,
    epochs = epochs,
    validation_data = raw_valid
)

Epoch 1/20


ValueError: in user code:

    File "c:\Users\Juanka\Documents\TFG\2-PrimerProgres\test_image_class\venv\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Juanka\Documents\TFG\2-PrimerProgres\test_image_class\venv\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Juanka\Documents\TFG\2-PrimerProgres\test_image_class\venv\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Juanka\Documents\TFG\2-PrimerProgres\test_image_class\venv\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Juanka\Documents\TFG\2-PrimerProgres\test_image_class\venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise